># Universidad Autónoma de Aguascalientes
>## *Ingeniería en Computación Inteligente*
>#### Materia:
>Inteligencia Artificial
>#### Topico: 
>Machine Learning (Agosto_Diciembre-2023)
>#### Integrantes del Equipo:
>- Juan Francisco Gallo Ramírez
>- José Alfredo Díaz Robledo
>- Luis Palbo Esparza Terrones
>- Luis Manuel Flores Jiménez 
>#### Maestro: 
>Dr. Francisco Javier Luna Rosas
>#### Fecha: 
>Octubre del 2023

# <<< Programa de Análisis de Sentimientos con Machine Learning >>>

## Importamos las librerías necesarias:

In [13]:
import os
import numpy  as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from pandas import DataFrame
import textprocess as t
from gensim.models import Word2Vec

## Lectura del archivo:
>Cargamos los datos del archivo, es este caso serán las reseñas de películas y la calfiicación de la reseña (positiva o negativa)

In [2]:
data = pd.read_excel("movie_data.xlsx")
reseñas = data['review'].copy()
sentimiento = data['sentiment'].copy()
print(">> Se muestran las reseñas a procesar:\n")
print(reseñas)
print("\n>> Se muestran la calificación de la reseña:\n")
print(sentimiento)

>> Se muestran las reseñas a procesar:

0        In 1974, the teenager Martha Moxley (Maggie Gr...
1        OK... so... I really like Kris Kristofferson a...
2        ***SPOILER*** Do not read this, if you think a...
3        hi for all the people who have seen this wonde...
4        I recently bought the DVD, forgetting just how...
                               ...                        
49995    OK, lets start with the best. the building. al...
49996    The British 'heritage film' industry is out of...
49997    I don't even know where to begin on this one. ...
49998    Richard Tyler is a little boy who is scared of...
49999    I waited long to watch this movie. Also becaus...
Name: review, Length: 50000, dtype: object

>> Se muestran la calificación de la reseña:

0        1
1        0
2        0
3        1
4        0
        ..
49995    0
49996    0
49997    0
49998    0
49999    1
Name: sentiment, Length: 50000, dtype: int64


## Preprocesamiento de texto:
>Aplicamos el procesamiento de texto, que consta de remover las contracciones, limpiar el texto de caracteres no válidos, remover palabras duplicadas, remover las palabras que carezcan de significado (stopwords), lemantizar el texto y finalmente tokenizar la reseña:

In [3]:
for i in range(len(reseñas)):
    reseñas[i] = t.remove_contractions(reseñas[i])
    reseñas[i] = t.clean_text(reseñas[i])
    reseñas[i] = t.remove_duplicates(reseñas[i])
    reseñas[i] = t.remove_stopwords(reseñas[i])
    reseñas[i] = t.lemmatize_text(reseñas[i])
    reseñas[i] = t.tokenize_text(reseñas[i])
print(">> Se muestran las reseñas procesadas:\n")
print(reseñas)

>> Se muestran las reseñas procesadas:

0        [teenager, martha, moxley, maggie, grace, move...
1        [ok, really, like, kris, kristofferson, usual,...
2        [spoiler, read, think, watching, movie, althou...
3        [hi, people, seen, wonderful, movie, im, sure,...
4        [recently, bought, dvd, forgetting, much, hate...
                               ...                        
49995    [ok, let, start, best, building, although, har...
49996    [british, heritage, film, industry, control, t...
49997    [even, know, begin, one, family, worst, line, ...
49998    [richard, tyler, little, boy, scared, everythi...
49999    [waited, long, watch, movie, also, like, bruce...
Name: review, Length: 50000, dtype: object


## Procesamiento de lenguaje natural (NLP) :
>Utilizamos Word2Vec para para representar palabras como vectores numéricos en un espacio multidimensional. Su principal objetivo es capturar las relaciones semánticas y contextuales entre palabras en un corpus de texto.

In [4]:
model = Word2Vec(sentences=reseñas, vector_size=100, window=5, min_count=1, sg=0)
def average_vector(tokens, model, vector_size):
    if tokens:
        vectors = [model.wv[word] for word in tokens if word in model.wv]
        if vectors:
            return sum(vectors) / len(vectors)
    return [0] * vector_size
reseñas = [average_vector(tokens, model, 100) for tokens in reseñas]

## Se muestran los resultados de nuestras matrices:
>Se convierten el su repectivo tipo de dato para ingresarlos a la red neuronal, además de imprimirlos.

In [5]:
X = np.array(reseñas)
print(">> Se muestra la matriz categorica:\n")
print(X)
y = sentimiento.values
print("\n>> Se muestra la matriz a predecir:\n")
print(y)

>> Se muestra la matriz categorica:

[[-0.14446671  0.47224125 -0.04878595 ... -0.05316269 -0.24201994
   0.0127915 ]
 [ 0.05499815 -0.10298393 -0.47050315 ...  0.26607272  0.3566507
   0.05657729]
 [-0.08200777  0.04941026 -0.316205   ...  0.19726215  0.2912637
   0.00336045]
 ...
 [ 0.14596942 -0.05613727 -0.41993722 ...  0.2573617   0.20607127
  -0.17194603]
 [ 0.00309675  0.06918902 -0.68949103 ...  0.11016285  0.10793357
   0.0392578 ]
 [-0.43930748 -0.25208524 -0.72263515 ... -0.00103745  0.44321424
   0.31891558]]

>> Se muestra la matriz a predecir:

[1 0 0 ... 0 0 1]


## Red neuronal:

>- Se separan los datos con el 70% de los datos para entrenamiento y el 30% para testing:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

>- Mediante el constructor inicializa la instancia_red, con el sovel 'adam':

In [7]:
instancia_red = MLPClassifier(solver='adam', max_iter=1000, random_state=0)
print(instancia_red)
instancia_red.fit(X_train,y_train)

MLPClassifier(max_iter=1000, random_state=0)


MLPClassifier(max_iter=1000, random_state=0)

>- Las predicciones del testing se muestran:

In [8]:
print(">> Las predicciones en Testing son: {}".format(instancia_red.predict(X_test)))

>> Las predicciones en Testing son: [1 0 1 ... 0 1 0]


>- Función para calcular los índices de calidad de la predicción:

In [9]:
def indices_general(MC, nombres = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    return {">> Matriz de Confusión":MC, 
            ">> Precisión Global":precision_global, 
            ">> Error Global":error_global}

>#### Índices de Calidad del Modelo:

In [10]:
prediccion = instancia_red.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


>> Matriz de Confusión:
[[6188 1154]
 [1285 6373]]

>> Precisión Global:
0.8374

>> Error Global:
0.16259999999999997
